In [ ]:
# !pip install transformers, tensorflow, torch

In [1]:
import pandas as pd
import warnings   
from transformers import (
    AutoTokenizer,
    BertTokenizer, 
    BertForSequenceClassification,
    pipeline,
)
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

In [2]:
finbert_pt_br_tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbert_pt_br_model = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")

finbert_pt_br_pipeline = pipeline(task='text-classification', model=finbert_pt_br_model, tokenizer=finbert_pt_br_tokenizer)

In [4]:
finbert_tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

finbert_pipeline = pipeline(task='text-classification', model=finbert_model, tokenizer=finbert_tokenizer)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [16]:
atuais = pd.read_csv('../data/news/noticias_atuais.csv')
noticias = pd.read_csv('../data/news/noticias_sentimentos.csv')
atuais['Data'] = pd.to_datetime(atuais['Data'], format='%Y-%m-%d')

atuais = atuais[~atuais['Notícia'].isin(noticias['Notícia'])]

news_list = atuais['Notícia'].tolist()
print(len(news_list))
news_list[:5]

166


['Ibovespa caiu 1,40% no último fechamento',
 'Volta dos IPOs deve ficar só para 2025, nas previsões mais otimistas',
 'resultado da loteria federal do 1 ao 10',
 'Dividendos hoje: JHSF (JHSF3) distribui 20 milhões em proventos; veja valor por ação – Investimentos – Estadão E-Investidor – As principais notícias do mercado financeiro',
 'Mais de 80% da bolsa de Pequim teve lucro no 1º semestre']

In [26]:
sentiments = pd.DataFrame(finbert_pt_br_pipeline(news_list))
sentiments

,label,score
0,NEUTRAL,0.613730
1,NEUTRAL,0.569116
2,NEUTRAL,0.577979
3,NEUTRAL,0.763031
4,NEGATIVE,0.831016
5,POSITIVE,0.762737
6,NEUTRAL,0.652559
7,POSITIVE,0.844132
8,POSITIVE,0.674256
9,NEGATIVE,0.827452


In [18]:
assert len(sentiments) == len(atuais), "Sentiments and news must have the same length"
atuais['Sentimento'] = sentiments['label']
atuais['Score'] = sentiments['score']

mercado_sentimentos = pd.read_csv('../data/news/noticias_sentimentos.csv')
mercado_sentimentos['Data'] = pd.to_datetime(mercado_sentimentos['Data'], format='%Y-%m-%d')
sentimentos = pd.concat([mercado_sentimentos, atuais], ignore_index=True)
assert len(sentimentos) == len(mercado_sentimentos) + len(atuais), "Sentiments and news must have the same length"
# sentimentos.to_csv('../data/news/noticias_sentimentos.csv', index=False)

,Data,Notícia,Sentimento,Score
84806,2024-09-08,"BMO Capital reafirma Market Perform para ações da Samsara, vê oportunidades \nde expansão",POSITIVE,0.731023
84694,2024-09-08,Foto do arquivo: fornecida por 【aniversario cassino】,NEUTRAL,0.512535
84734,2024-09-08,"Dividendos de até 15,24% são destaque no Money Times; veja as principais \nmanchetes dos jornais hoje (08)",POSITIVE,0.755705
84733,2024-09-08,baccarat jogo,NEGATIVE,0.594870
84732,2024-09-08,Agenda econômica mira índices de inflação no Brasil e EUA,POSITIVE,0.580710
...,...,...,...,...
30476,2014-09-08,VC no Como Será? - Letícia Queiroz responde perguntas enviadas pelo site do \r\nComo Será?,NEUTRAL,0.679012
30477,2014-09-08,Meu salário é de R$ 20 mil; como investir para me aposentar aos 45 com \r\nrenda de R$ 15 mil?,NEUTRAL,0.717937
30478,2014-09-08,» AMAZONAS ATUAL - Cartões pré-pagos avançam no mercado,POSITIVE,0.697899
30479,2014-09-08,Loja de roupas para executivas fatura R$ 600 mil em um ano e mira franquias,POSITIVE,0.721196


In [18]:
# Comparação FinBERT vs FinBERT-PT-BR

# FinBERT
finbert_outputs = pd.DataFrame(finbert_pipeline(news_list[:10]))
finbert_outputs['news'] = news_list[:10]

# FinBER-PT-BR
finbert_pt_br_outputs = pd.DataFrame(finbert_pt_br_pipeline(news_list[:10]))
finbert_pt_br_outputs['news'] = news_list[:10]

# Exibindo resultados
display(
    finbert_outputs,
    finbert_pt_br_outputs,
)

,label,score,news
0,positive,0.766157,Unipar e MELVER estão com inscrições abertas para MBA em Mercado Financeiro
1,positive,0.625554,"Mercado financeiro hoje: Vale (VALE3) em busca de parceiro, Azul (AZUL4) em \napuros e acordo da Weg (WEGE3) de R$ 630 milhões – Mercado – Estadão \nE-Investidor – As principais notícias do mercado financeiro"
2,neutral,0.848194,Briga entre X e STF pode afastar investidores do Brasil? Especialistas \ndivergem
3,neutral,0.947857,Podcast | Fechamento de Mercado - 03/09/2024
4,neutral,0.935288,Comentario semanal - Perspectivas
5,neutral,0.865317,Apagão cibernético global causa tensão no mercado financeiro
6,positive,0.652962,Confira dez tendências que o mercado financeiro deve incorporar nos \npróximos anos
7,neutral,0.850085,Mercados financeiros hoje: tom negativo em NY marca volta de feriado
8,positive,0.770710,Políticas do governo federal fazem PIB crescer acima do esperado e rumar \npara 3% ao final do ano
9,positive,0.570883,Setor financeiro ganha destaque nas indicações de ações para setembro


,label,score,news
0,NEUTRAL,0.700441,Unipar e MELVER estão com inscrições abertas para MBA em Mercado Financeiro
1,NEGATIVE,0.515265,"Mercado financeiro hoje: Vale (VALE3) em busca de parceiro, Azul (AZUL4) em \napuros e acordo da Weg (WEGE3) de R$ 630 milhões – Mercado – Estadão \nE-Investidor – As principais notícias do mercado financeiro"
2,NEGATIVE,0.850306,Briga entre X e STF pode afastar investidores do Brasil? Especialistas \ndivergem
3,NEUTRAL,0.614209,Podcast | Fechamento de Mercado - 03/09/2024
4,NEUTRAL,0.610426,Comentario semanal - Perspectivas
5,NEGATIVE,0.848840,Apagão cibernético global causa tensão no mercado financeiro
6,NEUTRAL,0.725723,Confira dez tendências que o mercado financeiro deve incorporar nos \npróximos anos
7,NEGATIVE,0.821666,Mercados financeiros hoje: tom negativo em NY marca volta de feriado
8,NEGATIVE,0.554580,Políticas do governo federal fazem PIB crescer acima do esperado e rumar \npara 3% ao final do ano
9,POSITIVE,0.519682,Setor financeiro ganha destaque nas indicações de ações para setembro
